<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/python_rests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# data clean 

BathroomsFull
BathroomsHalf
BathroomsTotalDecimal
BathroomsTotalInteger
BedroomsTotal
BuildingAreaTotal
City
CloseDate
ClosePrice
CumulativeDaysOnMarket
DaysOnMarket
GarageSpaces
Latitude
ListingContractDate
ListingId
ListPrice
ListSource
Longitude
LotSizeAcres
MlsStatus
OwnerName
OwnerPhone
OwnerPhoneAlternative
ParcelNumber
ParcelNumber2
PoolYN
PostalCode
PreviousListPrice
PreviousStatus
PropertySubType
PropertyType
PublicRemarks
PurchaseContractDate
SchoolDistrict
StandardStatus
StatusChangeTimestamp
StreetDirPrefix
StreetDirSuffix
StreetName
StreetNumber
StreetNumberNumeric
StreetSuffix
SubdivisionName
USProperty_MUI
YearBuilt



In [2]:
#import package
!pip install censusgeocode
import csv
import pandas as pd
import censusgeocode as cg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 992 kB/s 
     |████████████████████████████████| 54 kB 2.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


# Data cleaning

In [26]:
data= pd.read_csv("final.csv")
# select the data columns we want 
data = data[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt','LivingArea']]


In [27]:
# change data type
zip = []
lo = []
la = []

for item in data['PostalCode']:
    item = str(item)
    zip.append(item)
for item in data['Longitude']:
    item = float(item)
    lo.append(item)
for item in data['Latitude']:
    item = float(item)
    la.append(item)
data['PostalCode'] = zip
data['Longitude'] = lo
data['Latitude'] = la

In [ ]:
# choose columns status

data.loc[data['PropertySubType'] == 'Single Family Residence' ]
data.loc[data['PropertyType'] == 'Residential']

# Set Input

In [32]:
# input data address
print('Enter #MLS')
mls = 14730621
print('Enter address')
address = '2836 Meadowbrook Drive'
print('Enter city')
city = 'Dallas'
print('Enter state')
state = 'Texas'
print('Enter zipcode')
zip = '75227'



Enter #MLS
Enter address
Enter city
Enter state
Enter zipcode


In [40]:
# find value in csv 
res_mls = data.isin([mls]).any().any()
if res_mls :
  print("\nThis value exists in Dataframe")
  # get whole row value
  pull_value = data.loc[data['ListingId'] == mls]
  print(pull_value)

else :
  print("\nThis value does not exists in Dataframe")


This value exists in Dataframe
   BathroomsFull  BathroomsHalf  BathroomsTotalDecimal  BathroomsTotalInteger  \
1              3              0                    3.0                      3   

   BedroomsTotal  BuildingAreaTotal   City CloseDate  ClosePrice  \
1              5             2632.0  Plano   1/26/22    637000.0   

   CumulativeDaysOnMarket  ...  StreetDirPrefix  StreetDirSuffix   StreetName  \
1                       6  ...              NaN              NaN  Meadowbrook   

  StreetNumber  StreetNumberNumeric  StreetSuffix        SubdivisionName  \
1         2836                 2836         Drive  Prairie Creek Estates   

   USProperty_MUI  YearBuilt LivingArea  
1       411548787       1976       2632  

[1 rows x 46 columns]


In [105]:
# # limit zipcode range
data = data.loc[data['PostalCode'] == zip ]

In [98]:
# if there is no longitude 
#get Longitude and Latitude

##cg = CensusGeocode(benchmark='Public_AR_Current', vintage='Current_Current')
# example: 1620 Yarborough Drive, city = Sherman, State = TX , zipcode = 75092


result = cg.address(address, city = city, state= state,zipcode = zip)
lo = result[0]['coordinates']['x']
la = result[0]['coordinates']['y']
print(result[0]['coordinates'])

{'x': -96.76240415323872, 'y': 33.00313878557192}


In [103]:
# limit longitude and latitude

new_data = data[(data['Longitude'] > lo - 0.01) & (data['Longitude'] < lo + 0.01)&
               (data['Latitude'] < la + 0.02)& (data['Latitude'] > la - 0.02)]

In [ ]:
# limit the days

data = data[data['CumulativeDaysOnMarket'].isin(range(0,90))] 

In [ ]:
# limit square feet

if 'LivingArea' <= 1100:
  data[(data['LivingArea'] > area - 100) & (data['LivingArea'] < area + 100)
elif 'LivingArea' <= 2200:
  data[(data['LivingArea'] > area -200 )& (data['LivingArea'] > area + 200)]
else:
  data[(data['LivingArea'] > area -300 )& (data['LivingArea'] > area + 300)]


In [ ]:
# check number of house = (5,6)个 ok， 不等于repet 上面步骤扩大范围

In [ ]:
# get top 3 mean max min
# return csv